In [12]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

import cv2
import requests
import time
import serial  # << 추가

Ultralytics 8.3.223 🚀 Python-3.12.3 torch-2.9.0+cu128 CPU (13th Gen Intel Core i5-1335U)
Setup complete ✅ (12 CPUs, 15.3 GB RAM, 61.1/232.6 GB disk)


In [16]:
# -----------------------------
# 1) 설정값
# -----------------------------
MODEL_PATH = '/home/vit/dev_ws/project/Media-Project/ai_server/12_model.pt'
DATA_SERVER_URL = "http://192.168.0.104:5002/process_trash"  # 데이터 서버 주소

CAM_INDEX = 2          # 웹캠 번호
CONF_THRES = 0.25      # 최소 신뢰도

# 아두이노(초음파 센서) 시리얼 설정
SERIAL_PORT = "/dev/ttyUSB0"  # 우분투 기준. 필요하면 /dev/ttyUSB0 등으로 변경
BAUD_RATE = 9600
TRIGGER_DIST_CM = 7.0      # 7cm 이하일 때 트리거
TRIGGER_COOLDOWN_SEC = 2.0 # 연속 트리거 방지용 쿨다운

# YOLO 클래스 인덱스 -> 실제 라벨 이름 (학습 시 사용한 순서)
CLASS_NAMES = {
    0: "종이",
    1: "종이팩",
    2: "종이컵",
    3: "캔류",
    4: "유리병",
    5: "페트",
    6: "플라스틱",
    7: "비닐",
    8: "유리+다중포장재",
    9: "페트+다중포장재",
    10: "스티로폼",
    11: "건전지",
}

# YOLO 클래스 idx -> 데이터서버의 type_id 매핑
#   5, 6, 9 -> 플라스틱(type_id=2)
#   3       -> 캔(type_id=3)
#   나머지 -> 일반(type_id=1)
YOLO_TO_TYPE_ID = {
    0: 1,   # 종이 -> 일반
    1: 1,   # 종이팩 -> 일반
    2: 1,   # 종이컵 -> 일반
    3: 3,   # 캔류 -> 캔
    4: 1,   # 유리병 -> 일반
    5: 2,   # 페트 -> 플라스틱
    6: 2,   # 플라스틱 -> 플라스틱
    7: 1,   # 비닐 -> 일반
    8: 1,   # 유리+다중포장재 -> 일반
    9: 2,   # 페트+다중포장재 -> 플라스틱
    10: 1,  # 스티로폼 -> 일반
    11: 1,  # 건전지 -> 일반
}

# -----------------------------
# 2) YOLO 결과에서 클래스 하나 뽑기 (cls/ det 둘 다 대응)
# -----------------------------
def get_pred_label(results):
    """YOLOv8 결과에서 가장 확신 높은 클래스 idx와 confidence 반환"""
    r = results[0]

    # (1) 분류 모델(cls)인 경우
    if hasattr(r, "probs") and r.probs is not None:
        idx = int(r.probs.top1)
        conf = float(r.probs.top1conf)
        return idx, conf

    # (2) 탐지 모델(det)인 경우: 가장 conf 높은 박스 하나 선택
    if hasattr(r, "boxes") and r.boxes is not None and len(r.boxes) > 0:
        max_conf_idx = int(r.boxes.conf.argmax().item())
        idx = int(r.boxes.cls[max_conf_idx].item())
        conf = float(r.boxes.conf[max_conf_idx].item())
        return idx, conf

    return None, None

# -----------------------------
# 3) 데이터 서버 호출
# -----------------------------
def send_to_data_server(type_id: int):
    """데이터 서버 /process_trash 에 type_id 전송"""
    payload = {"type_id": type_id}
    try:
        print(f"[INFO] 데이터서버로 전송: {payload}")
        resp = requests.post(DATA_SERVER_URL, json=payload, timeout=5)
        print(f"[INFO] 응답 코드: {resp.status_code}, 응답 내용: {resp.text}")
    except Exception as e:
        print(f"[ERROR] 데이터서버 통신 실패: {e}")

# -----------------------------
# 4) 메인 루프: 웹캠 + 초음파 트리거
# -----------------------------
def main():
    print("[INFO] YOLO 모델 로딩 중...")
    model = YOLO(MODEL_PATH)
    print("[INFO] 모델 로딩 완료!")

    # 웹캠 오픈
    cap = cv2.VideoCapture(CAM_INDEX)
    if not cap.isOpened():
        print("[ERROR] 웹캠을 열 수 없습니다.")
        return

    # 시리얼 포트 오픈
    try:
        ser = serial.Serial(SERIAL_PORT, BAUD_RATE, timeout=0.05)  # 짧은 타임아웃
        print(f"[INFO] 시리얼 포트 열림: {SERIAL_PORT} @ {BAUD_RATE}")
    except Exception as e:
        print(f"[ERROR] 시리얼 포트를 열 수 없습니다: {e}")
        cap.release()
        return

    last_trigger_time = 0.0

    print("[INFO] 웹캠 + 초음파 트리거 시작!")
    print(" - 초음파 센서가 7cm 이하를 감지하면 자동으로 캡처 & 분류 & 데이터서버 전송")
    print(" - 'q' 를 누르면 종료")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("[ERROR] 프레임을 읽을 수 없습니다.")
            break

        # 초음파 센서로부터 시리얼 데이터 읽기
        line = ser.readline().decode(errors="ignore").strip()
        triggered = False
        dist_cm = None

        if line:
            # 1) 숫자라면 거리(cm) 값으로 가정
            try:
                dist_cm = float(line)
                # 너무 말도 안 되는 값(노이즈) 필터링
                if 0 < dist_cm < 200:  # 2m 이내만 유효
                    # 화면에 거리 찍어보면 디버깅에 도움됨
                    # print(f"[SERIAL] distance = {dist_cm:.1f} cm")
                    if dist_cm <= TRIGGER_DIST_CM:
                        triggered = True
            except ValueError:
                # 2) 숫자가 아니면, "DETECT" 같은 트리거 문자열로 가정
                if line.upper() == "DETECT":
                    triggered = True

        # 현재 화면에 안내 문구 표시
        text = "Ultrasonic: waiting..."
        if dist_cm is not None:
            text = f"Distance: {dist_cm:.1f} cm"
        cv2.putText(frame, text,
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, "Press 'q' to quit",
                    (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow("DeepCycle AI Webcam (Ultrasonic Trigger)", frame)
        key = cv2.waitKey(1) & 0xFF

        # 종료 키
        if key == ord('q'):
            print("[INFO] 종료합니다.")
            break

        # 트리거 발생 시 + 쿨다운 체크
        now = time.time()
        if triggered and (now - last_trigger_time) > TRIGGER_COOLDOWN_SEC:
            last_trigger_time = now
            print("\n[INFO] ----- 초음파 트리거 감지! 캡처 & 분류 시작 -----")
            if dist_cm is not None:
                print(f"[INFO] 감지 거리: {dist_cm:.1f} cm")

            # YOLO 추론
            t0 = time.time()
            results = model.predict(source=frame, conf=CONF_THRES, verbose=False)
            infer_ms = (time.time() - t0) * 1000.0

            idx, conf = get_pred_label(results)
            if idx is None:
                print("[WARN] 객체를 인식하지 못했습니다.")
                continue

            label_kor = CLASS_NAMES.get(idx, f"cls_{idx}")
            type_id = YOLO_TO_TYPE_ID.get(idx, 1)  # 혹시 누락되면 기본 1(일반)

            print(
                f"[RESULT] YOLO idx={idx} "
                f"({label_kor}), type_id={type_id}, "
                f"conf={conf:.3f}, time={infer_ms:.1f}ms"
            )

            # 데이터 서버로 전송
            send_to_data_server(type_id)

    cap.release()
    ser.close()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


[INFO] YOLO 모델 로딩 중...
[INFO] 모델 로딩 완료!
[ERROR] 시리얼 포트를 열 수 없습니다: module 'serial' has no attribute 'Serial'
